# Cruce DMM a Beam

In [5]:
import pandas as pd

columns = ["ts", "IDC", "IAC"]
#file_path = r'..\0_raw\dmm_data6894024_2022_05_27_212453_all.dat' # este no calza con ningun beam data por los tiempos
file_path = r'..\0_raw\dmm_data_2022_06_02-14_all.dat'

#Abro el fichero, reemplazo los '\x00' y guardo en memoria
with open(file_path, 'r', encoding='latin-1', errors='ignore') as f:
    text = f.read().replace('\x00', '')

from io import StringIO
df = pd.read_csv(
    StringIO(text),
    delim_whitespace=True,
    names=columns,
    comment="#",
    engine="python"
)

# time stamps
df["Time"] = pd.to_datetime(df["ts"], unit="s") + pd.Timedelta(hours=2)
df = df.set_index("Time")

# Integridad
print(df.isna().sum())                     # Buscar NaN
print(df.index.to_series().diff().describe())  # Revisar intervalos de ~4 s


ts     0
IDC    0
IAC    0
dtype: int64
count                       273857
mean     0 days 00:00:04.248861707
std      0 days 00:00:30.843098908
min      0 days 00:00:04.079984896
25%      0 days 00:00:04.100334848
50%      0 days 00:00:04.106866176
75%      0 days 00:00:04.173525248
max      0 days 02:41:50.620609280
Name: Time, dtype: object


cruce con datos beam

In [6]:
# Aseguramos que 'df' tenga columna Time:
if df.index.name == "Time" or df.index.dtype.kind == "M":
    df_curr = df.reset_index()
else:
    df_curr = df.copy()
    if "Time" not in df_curr.columns:
        raise KeyError("df debe tener columna o índice llamado 'Time' en formato datetime.")
    df_curr["Time"] = pd.to_datetime(df_curr["Time"])

# Ordenar ambos DataFrames por Time
df_curr = df_curr.sort_values("Time").reset_index(drop=True)
beam_data_sorted = beam_data.sort_values("Time").reset_index(drop=True)

# Merge asof con tolerancia de 1 segundo
merged = pd.merge_asof(
    df_curr,
    beam_data_sorted,
    on="Time",
    direction="nearest",
    tolerance=pd.Timedelta(seconds=1)
)

# Verificar filas sin correspondencia (NaN en columnas de beam_data)
#    Suponemos que beam_data tiene columnas ["TID", "HEH", "N1MeV", "run_group"]
beam_cols = ["TID", "HEH", "N1MeV", "run_group"]
mask_missing = merged[beam_cols].isna().any(axis=1)

if mask_missing.any():
    bad_times = merged.loc[mask_missing, "Time"]
    raise ValueError(
        f"Error al fusionar: {mask_missing.sum()} filas de 'df' no encontraron registro de beam "
        f"dentro de 1 s.\nTimestamps sin match:\n{bad_times.tolist()}"
    )

# Si todo está bien, restauramos índice en Time (opcional) y mostramos resultado
merged = merged.set_index("Time")
print("Merge exitoso. DataFrame resultante:")
print(merged.head())

NameError: name 'beam_data' is not defined